<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель" data-toc-modified-id="Модель-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Модель</a></span></li></ul></li></ul></div>

# Проект для «Викишоп» c BERT 

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Imports

In [1]:
import pandas as pd
import torch
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression




In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Functions

## Подготовка

In [3]:
try:
    data  = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    print('Файл в папке отсутствует')
    data  = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)

Файл в папке отсутствует


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [6]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [7]:
data[data['text'].duplicated()]

,text,toxic


**Вывод** в данных отсутсвуют дубликаты пропуски, так же отсутствуют есть дисбаланс классов

## Обучение

данных много а памяти мало поэтому возьмем лишь часть данных , учитывая дизбалланс классов

In [8]:
features = data['text']
target = data['toxic']

In [9]:
features_reduced, features_stash, target_reduced, target_stash = train_test_split(features, target, 
                                                                      test_size=0.9, random_state=12345, stratify=target)

### Модель

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Загрузка модели и токенизатора
model_name = 'unitary/toxic-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Функция для предсказания токсичности комментария
def predict_toxicity(comment):
    inputs = tokenizer.encode_plus(comment, return_tensors='pt', padding=True, truncation=True, 
                                   max_length=512).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    preds = torch.sigmoid(logits).detach().cpu().numpy().flatten()
    # Классификация на основе порога 0.5
    if preds[0] > 0.5:
        return 1
    else:
        return 0

In [11]:
%%time
predicts_train = features_reduced.apply(predict_toxicity)

CPU times: total: 3min 9s
Wall time: 3min 12s


In [12]:
predicts_train

24301     0
36076     0
48450     1
42562     0
148397    0
         ..
135160    0
85772     1
46878     0
88576     0
103125    1
Name: text, Length: 15929, dtype: int64

In [13]:
f1 = f1_score(target_reduced, predicts_train)

In [14]:
print(f1)

0.936582318025617


**Вывод** мы получили отличную точность как и требовалось в ТЗ